In [6]:
import random
import psycopg2
from datetime import datetime
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Database connection setup
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")

# Establish the connection using psycopg2
conn = psycopg2.connect(
    dbname=db_name, user=db_user, password=db_password, host=db_host
)
print(f"Connected to database: {db_name}")

def get_cash_balance(conn, account_id):
    with conn.cursor() as cur:
        cur.execute("""
            SELECT available_cash_balance
            FROM public.vw_cash_balance
            WHERE account_id = %s
            order by random() desc
            LIMIT 250;
        """, (account_id,))
        result = cur.fetchone()
    return result[0] if result else 0

def get_asset_balance(conn, account_id):
    with conn.cursor() as cur:
        cur.execute("""
            SELECT account_id, asset_id, asset_holding
            FROM public.vw_asset_balance
            WHERE account_id = %s
            ORDER BY random() DESC
            LIMIT 250;
        """, (account_id,))
        result = cur.fetchone()
    return result if result else None

def get_random_account_for_buy(conn):
    with conn.cursor() as cur:
        cur.execute("""
            SELECT account_id
            FROM public.vw_cash_balance
            WHERE available_cash_balance > 0
            ORDER BY random() DESC
            LIMIT 250;
        """)
        return cur.fetchone()

def get_random_account_for_sell(conn):
    with conn.cursor() as cur:
        cur.execute("""
            SELECT account_id
            FROM public.vw_asset_balance
            WHERE asset_holding > 0
            ORDER BY random() DESC
            LIMIT 250;
        """)
        return cur.fetchone()

def get_asset_list(conn):
    with conn.cursor() as cur:
        cur.execute("""
            SELECT asset_id, latest_price, currency_code
            FROM public.vw_latest_price;
        """)
        return cur.fetchall()

def get_asset_price(conn, asset_id):
    with conn.cursor() as cur:
        cur.execute("""
            SELECT latest_price, currency_code
            FROM public.vw_latest_price
            WHERE asset_id = %s;
        """, (asset_id,))
        result = cur.fetchone()
    return result if result else None
def simulate_trade(conn, trade_count):
    try:
        if trade_count > 100:
            print("Trade limit reached.")
            return

        # Randomly choose trade type: Buy or Sell
        trade_type = random.choices(['Buy', 'Sell'], weights=[90, 10], k=1)[0]

        if trade_type == 'Buy':
            random_account = get_random_account_for_buy(conn)
            if random_account:
                account_id = random_account[0]
                cash_balance = float(get_cash_balance(conn, account_id))  # Convert Decimal to float

                if cash_balance <= 0:
                    print(f"Account {account_id} has insufficient cash balance (cash balance: {cash_balance}). Skipping.")
                    return

                assets = get_asset_list(conn)
                if not assets:
                    print(f"No assets available for trading for account {account_id}.")
                    return

                # Select a random asset to trade
                asset_id, asset_price, currency_code = random.choice(assets)
                asset_price = float(asset_price)  # Convert Decimal to float

                if not currency_code:
                    print(f"Error: Asset {asset_id} does not have a currency code. Skipping trade.")
                    return

                max_quantity = int(cash_balance // asset_price)  # Use int for random.randint
                print(f"Account {account_id} can afford {max_quantity} units of asset {asset_id}.")

                if max_quantity < 1:
                    print(f"Account {account_id} does not have enough balance to buy any asset.")
                    return

                trade_quantity = random.randint(1, max_quantity)
                trade_cost = trade_quantity * asset_price

                with conn.cursor() as cur:
                    cur.execute("""
                        INSERT INTO public.asset_trade (
                            account_id, asset_id, filled_units, filled_price, trade_type, trade_status, date_placed, date_created, date_updated, quote_units, quote_price
                        )
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, (account_id, asset_id, trade_quantity, asset_price, 'Buy', 'Completed', datetime.now(), datetime.now(), datetime.now(), trade_quantity, asset_price))

                trade_note = f"Bought {trade_quantity} units of {asset_id}"
                with conn.cursor() as cur:
                    cur.execute("""
                        INSERT INTO public.cash_trade (
                            account_id, amount, currency_code, trade_status, trade_note, date_created, date_updated
                        )
                        VALUES (%s, %s, %s, %s, %s, %s, %s)
                    """, (account_id, -trade_cost, currency_code, 'Completed', trade_note, datetime.now(), datetime.now()))

                conn.commit()

                print(f"Account {account_id} bought {trade_quantity} of asset {asset_id} at {asset_price} {currency_code}. Deducted {trade_cost} from cash.")

        elif trade_type == 'Sell':
            random_account = get_random_account_for_sell(conn)
            if random_account:
                account_id = random_account[0]
                asset_balance = get_asset_balance(conn, account_id)

                if asset_balance:
                    _, asset_id, asset_holding = asset_balance
                    asset_holding = int(asset_holding)  # Convert Decimal to int

                    asset_data = get_asset_price(conn, asset_id)
                    if asset_data:
                        asset_price, currency_code = asset_data
                        asset_price = float(asset_price)  # Convert Decimal to float
                    else:
                        print(f"Error: No price or currency found for asset {asset_id}. Skipping sell.")
                        return

                    trade_quantity = random.randint(1, asset_holding)
                    trade_cost = trade_quantity * asset_price

                    with conn.cursor() as cur:
                        cur.execute("""
                            INSERT INTO public.asset_trade (
                                account_id, asset_id, filled_units, filled_price, trade_type, trade_status, date_placed, date_created, date_updated, quote_units, quote_price
                            )
                            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                        """, (account_id, asset_id, -trade_quantity, asset_price, 'Sell', 'Completed', datetime.now(), datetime.now(), datetime.now(), trade_quantity, asset_price))

                    trade_note = f"Sold {trade_quantity} units of {asset_id}"
                    with conn.cursor() as cur:
                        cur.execute("""
                            INSERT INTO public.cash_trade (
                                account_id, amount, currency_code, trade_status, trade_note, date_created, date_updated
                            )
                            VALUES (%s, %s, %s, %s, %s, %s, %s)
                        """, (account_id, trade_cost, currency_code, 'Completed', trade_note, datetime.now(), datetime.now()))

                    conn.commit()

                    print(f"Account {account_id} sold {trade_quantity} of asset {asset_id} at {asset_price}. Added {trade_cost} to cash.")

    except Exception as e:
        print(f"Error simulating trade: {e}")


# Example usage: Simulate trades for 15 random accounts
for i in range(500):  # Limit to 15 trades
    simulate_trade(conn, i)

conn.close()

Connected to database: vestia
Account 87230 can afford 105 units of asset 34.
Account 87230 bought 37 of asset 34 at 37.88 USD. Deducted 1401.5600000000002 from cash.
Account 86929 can afford 193 units of asset 11.
Account 86929 bought 161 of asset 11 at 23.52 USD. Deducted 3786.72 from cash.
Account 86840 can afford 4 units of asset 85.
Account 86840 bought 4 of asset 85 at 115.64 USD. Deducted 462.56 from cash.
Account 86930 can afford 16 units of asset 24.
Account 86930 bought 15 of asset 24 at 210.08 USD. Deducted 3151.2000000000003 from cash.
Account 87778 can afford 9 units of asset 27.
Account 87778 bought 1 of asset 27 at 208.58 USD. Deducted 208.58 from cash.
Account 87666 can afford 5 units of asset 61.
Account 87666 bought 4 of asset 61 at 180.78 USD. Deducted 723.12 from cash.
Account 87490 can afford 16 units of asset 37.
Account 87490 bought 8 of asset 37 at 237.44 USD. Deducted 1899.52 from cash.
Account 88705 can afford 3 units of asset 27.
Account 88705 bought 1 of ass